# Home Assignment 3 - Natural Language Processing (Majid Sohrabi)

In [ ]:
pip install opencorpora-tools

In [ ]:
import nltk
import random
import opencorpora
from sklearn.metrics import accuracy_score, recall_score, precision_score
import matplotlib.pyplot as plt
plt.style.use('ggplot')

# Task 3
from nltk import load_parser

# Task 4
from nltk.sem.drt import DrtParser

## Task 1
  1. Translate into predicate logic:
* a. Angus likes Cyril and Irene hates Cyril.
* b. Bruce loves himself and Pat does too.
* c. Cyril saw Bertie, but Angus didn't.
* d. Cyril is a fourlegged friend.
* e. Angus likes someone and someone likes Julia.
* f. Angus loves a dog who loves him.
* g. Nobody coughed or sneezed.
* h. Cyril likes everyone except for Irene.

In [ ]:
# a. Predicate logic for "Angus likes Cyril and Irene hates Cyril." expression
nltk.Expression.fromstring("like(Angus, Cyril) & hate(Irene, Cyril)")

# b. Predicate logic for "Bruce loves himself and Pat does too." expression
nltk.Expression.fromstring("love(Bruce, Bruce) & hate(Pat, Pat)")

# c. Predicate logic for "Cyril saw Bertie, but Angus didn't." expression
nltk.Expression.fromstring("saw(Cyril, Bertie) & !saw(Angus, Bertie)")

# d. Predicate logic for "Cyril is a fourlegged friend." expression
nltk.Expression.fromstring("has_four_legs(Cyril) & friendly(Cyril)")

# e. Predicate logic for "Angus likes someone and someone likes Julia." expression
nltk.Expression.fromstring("(exists x. like(Angus, x)) & (exists y. like(y, Julia))")

# f. Predicate logic for Angus loves a dog who loves him."" expression
nltk.Expression.fromstring("exists x. (love(Angus, x) & love(x, Angus))")

# g. Predicate logic for "Nobody coughed or sneezed." expression
nltk.Expression.fromstring("all x. - (cough(x) | sneeze(x))")

# h. Predicate logic for "Cyril likes everyone except for Irene." expression
nltk.Expression.fromstring("all x. (x != Irene -> like(Cyril, x))")

<AllExpression all x.(-(x = Irene) -> like(Cyril,x))>

## Task 2
  2. Translate into lambda expressions over a predicate logic formulas:
* a. feed Cyril and give a capuccino to Angus
* b. be given a book by Pat
* c. be loved or detested by everyone
* d. be loved by everyone and detested by no-one

In [ ]:
expression_reader = nltk.sem.Expression.fromstring

# a. Translate into lambda expressions over a predicate logic formulas: "feed Cyril and give a capuccino to Angus" expression
first_expre = expression_reader(r'\x.(feed(x, cyril) & exists y.(cappuccino(y) & give(x, y, angus)))').simplify()

# b. Translate into lambda expressions over a predicate logic formulas: "be given a book by Pat" expression
second_expre = expression_reader(r'\x.exists y.(book(y) & give(pat, y, x))').simplify()

# c. Translate into lambda expressions over a predicate logic formulas: "be loved or detested by everyone" expression
third_expre = expression_reader(r'\x. all y.(love(y, x) | detest(y, x))').simplify()

# d. Translate into lambda expressions over a predicate logic formulas: "be loved by everyone and detested by no-one" expression
fourth_expre = expression_reader(r'\x. all y.(love(y, x) & -detest(y, x))').simplify()

In [ ]:
first_expre

<LambdaExpression \x.(feed(x,cyril) & exists y.(cappuccino(y) & give(x,y,angus)))>

In [ ]:
second_expre

<LambdaExpression \x.exists y.(book(y) & give(pat,y,x))>

In [ ]:
third_expre

<LambdaExpression \x.all y.(love(y,x) | detest(y,x))>

In [ ]:
fourth_expre

<LambdaExpression \x.all y.(love(y,x) & -detest(y,x))>

## Task 3
  3. For the following discourse create a grammar which translates the sentences into predicate logic. Manually create a model to check the truth of the resulting formulas. 
* a. A black kitten saw a mouse. The kitten rushed after it.
The mouse jumped into a jar [of flour].
The kitten followed it.
The mouse ran away.
A white kitten got out of the jar.

  You can omit the part in the square brackets if there is a problem with it. Treat the article 'the' as the quantifier 'exists unique'. The lambda expression for it would be \P Q.exists x.((P(x) & Q(x)) & all y.(P(y) -> (x = y))). If there is a problem with the pronoun 'it', you can replace it with the noun phrase 'the mouse' directly in the text and process the modified discourse.

  Take the following grammar as an example:
nltk.data.show_cfg('grammars/book_grammars/discourse.fcfg')

In [ ]:
nltk.download('book_grammars')
nltk.data.show_cfg("Task3_grammar.fcfg")

[nltk_data] Downloading package book_grammars to /root/nltk_data...
[nltk_data]   Package book_grammars is already up-to-date!
% start S
# Grammar Rules
S[SEM = <app(?subj,?vp)>] -> NP[NUM=?n,SEM=?subj] VP[NUM=?n,SEM=?vp]
NP[NUM=?n,SEM=<app(?det,?nom)> ] -> Det[NUM=?n,SEM=?det]  Nom[NUM=?n,SEM=?nom]
NP[LOC=?l,NUM=?n,SEM=?np] -> PropN[LOC=?l,NUM=?n,SEM=?np]
NP[-LOC,NUM=sg,SEM=<\Q. (- exists x. (person(x) & Q(x)))>] -> 'nobody' | 'Nobody'
NP[-LOC,NUM=sg,SEM=<\Q. exists x. (person(x) & Q(x))>] -> 'somebody' | 'Somebody'
Pred[SEM=?prd] -> PredN[SEM=?prd] | PP[+LOC,+PRED,SEM=?prd] | Adj[SEM=?prd]
PredN[NUM=?n, SEM=?nom] -> Det[NUM=?n] Nom[NUM=?n, SEM=?nom]
Nom[NUM=?n,SEM=?nom] -> N[NUM=?n,SEM=?nom]
Nom[NUM=?n,SEM=<app(?pp,?nom)>] -> N[NUM=?n,SEM=?nom] PP[SEM=?pp]
Nom[NUM=?n,SEM=<?adj(?nom)>] -> Adj[SEM=?adj] N[NUM=?n,SEM=?nom]
VP[NUM=?n,SEM=<app(?v,?obj)>] -> TV[NUM=?n,SEM=?v] NP[SEM=?obj]
VP[NUM=?n,SEM=<app(?v,?p)>] -> IV[NUM=?n,SEM=?v] P[LOC=?l,SEM=?p]
VP[NUM=?n,SEM=?v] -> IV[NUM=?n,SEM=?

In [ ]:
mmy_grammar = """

% start S

# GRAMMAR RULES

S[SEM = <app(?subj,?vp)>] -> NP[NUM=?n,SEM=?subj] VP[NUM=?n,SEM=?vp]
NP[NUM=?n,SEM=<app(?det,?nom)> ] -> Det[NUM=?n,SEM=?det]  Nom[NUM=?n,SEM=?nom]
Nom[NUM=?n,SEM=?nom] -> N[NUM=?n,SEM=?nom]
Nom[NUM=?n,SEM=<?adj(?nom)>] -> Adj[SEM=?adj] N[NUM=?n,SEM=?nom] 
VP[NUM=?n,SEM=<app(?v,?obj)>] -> TV[NUM=?n,SEM=?v] NP[SEM=?obj]
VP[NUM=?n,SEM=?v] -> IV[NUM=?n,SEM=?v]


# LEXICAL RULES

# Articles
Det[NUM=sg,SEM=<\\P Q.exists x.(P(x) & Q(x))>] -> 'a' | 'A'
Det[NUM=sg,SEM=<\\P Q.exists x.((P(x) & Q(x)) & all y.(P(y) -> (x = y)))>] -> 'the' | 'The' 

# Nouns
N[NUM=sg,SEM=<\\x.kitten(x)>] -> 'kitten'
N[NUM=sg,SEM=<\\x.mouse(x)>] -> 'mouse' 
N[NUM=sg,SEM=<\\x.jar(x)>] -> 'jar'

# Adjectives
Adj[SEM=<\\P x.(white(x) & P(x))>] -> 'white'
Adj[SEM=<\\P x.(black(x) & P(x))>] -> 'black'

# Verbs
TV[SEM=<\\X y.X(\\x.see(y,x))>,tns=past] -> 'saw'
TV[SEM=<\\X y.X(\\x.rush_after(y,x))>,tns=past] -> 'rushed' 'after'
TV[SEM=<\\X y.X(\\x.jump_into(y,x))>,tns=past] -> 'jumped' 'into'
TV[SEM=<\\X y.X(\\x.follow(y,x))>,tns=past] -> 'followed'
IV[SEM=<\\x.run_away(x)>,tns=past] -> 'ran' 'away'
TV[SEM=<\\X y.X(\\x.get_out_of(y,x))>,tns=past] -> 'got' 'out' 'of'
"""

In [ ]:
f = open('mmy_grammar.fcfg', 'a')
for line in mmy_grammar:
  f.write(line)
f.close()

parser = load_parser('mmy_grammar.fcfg', trace = 0)

Lets define sentenses

In [ ]:
target_sentences = ['A black kitten saw a mouse',
                    'The kitten rushed after the mouse',
                    'The mouse jumped into a jar',
                    'The kitten followed the mouse',
                    'The mouse ran away',
                    'A white kitten got out of the jar']

In [ ]:
dom = {'k', 'm', 'j'}
target_ch = """

kitten => {k}
mouse => {m}
jar => {j}

black => {k}
white => {k}

see => {(k, m)}
rush_after => {(k, m)}
jump_into => {(m, j)}
follow => {(k, m)}
run_away => {m}
get_out_of => {(k, j)}
"""

Valuat = nltk.Valuation.fromstring(target_ch)
assign = nltk.Assignment(dom)
mod = nltk.Model(dom, Valuat)

'''
tsentence_ch = zip(target_sentences, target_ch)
for tsent, ch in tsentence_ch :
  print("The Target Sentence Is: ```", tsent, "```")
  Valuat = nltk.Valuation.fromstring(ch)
  print(nltk.evaluate_sents([tsent], 'Task3_grammar.fcfg', mod, assign))
  print('\n\n')

print('*' * 100, '\n')
'''

for tsent in target_sentences :
  print("The Target Sentence Is: ```", tsent, "```")
  tokens = tsent.split()
  
  for mytree in parser.parse(tokens):
    print(nltk.evaluate_sents([tsent], 'Task3_grammar.fcfg', mod, assign))
    print('\n\n')

The Target Sentence Is: ``` A black kitten saw a mouse ```
[[(Tree(S[SEM=<exists x.(black(x) & kitten(x) & exists z411.(mouse(z411) & see(x,z411)))>], [Tree(NP[NUM='sg', SEM=<\Q.exists x.(black(x) & kitten(x) & Q(x))>], [Tree(Det[NUM='sg', SEM=<\P Q.exists x.(P(x) & Q(x))>], ['A']), Tree(Nom[NUM='sg', SEM=<\x.(black(x) & kitten(x))>], [Tree(Adj[SEM=<\P x.(black(x) & P(x))>], ['black']), Tree(N[NUM='sg', SEM=<\x.kitten(x)>], ['kitten'])])]), Tree(VP[NUM=?n, SEM=<\y.exists x.(mouse(x) & see(y,x))>], [Tree(TV[SEM=<\X y.X(\x.see(y,x))>, tns='past'], ['saw']), Tree(NP[NUM='sg', SEM=<\Q.exists x.(mouse(x) & Q(x))>], [Tree(Det[NUM='sg', SEM=<\P Q.exists x.(P(x) & Q(x))>], ['a']), Tree(Nom[NUM='sg', SEM=<\x.mouse(x)>], [Tree(N[NUM='sg', SEM=<\x.mouse(x)>], ['mouse'])])])])]), <ExistsExpression exists x.(black(x) & kitten(x) & exists z411.(mouse(z411) & see(x,z411)))>, True)]]



The Target Sentence Is: ``` The kitten rushed after the mouse ```
[[(Tree(S[SEM=<exists x.(kitten(x) & exists z415.(

## Task 4
  4. The same as in the previous task but instead of predicate logic formulas translate the sentences into DRSs. Treat the article 'the' the same way as 'a', but in addition mark the variable with PRO predicate to show that it requires resolution in the same way as pronouns. Combine all DRSs into one using '+' operation and manually add resolution for all PRO variables. I. e. add conditions of the form (x = y), where x is the variable marked with PRO and y is its antecedent (the object which the pronoun or the noun phrase refers to). You can add them by creating a separate DRS and combining it with the rest of DRSs using '+' operation. After that use simplify() to get one DRS as a result of 'addition', then eliminate_equality() to reduce the number of variables, the fol() to convert the resulting DRS into predicate logic and check the truth of the formula in your model.

  Take the following grammar as an example:
nltk.data.show_cfg('grammars/book_grammars/drt.fcfg')

In [ ]:
def parse_p(parser, sentence):
    tokens = sentence.split()
    for tree in parser.parse(tokens):
        return(tree.label()['SEM'])

In [ ]:
nltk.data.show_cfg('Task4_DRS.fcfg')

% start S
# Grammar Rules
S[SEM = <app(?subj,?vp)>] -> NP[NUM=?n,SEM=?subj] VP[NUM=?n,SEM=?vp]
NP[NUM=?n,SEM=<app(?det,?nom)> ] -> Det[NUM=?n,SEM=?det]  Nom[NUM=?n,SEM=?nom]
NP[LOC=?l,NUM=?n,SEM=?np] -> PropN[LOC=?l,NUM=?n,SEM=?np]
Nom[NUM=?n,SEM=?nom] -> N[NUM=?n,SEM=?nom]
Nom[NUM=?n,SEM=<app(?pp,?nom)>] -> N[NUM=?n,SEM=?nom] PP[SEM=?pp]
Nom[NUM=?n,SEM=<app(?adj,?nom)>] -> Adj[SEM=?adj] N[NUM=?n,SEM=?nom]
VP[NUM=?n,SEM=?v] -> IV[NUM=?n,SEM=?v]
VP[NUM=?n,SEM=<app(?v,?obj)>] -> TV[NUM=?n,SEM=?v] NP[SEM=?obj]
# Lexical Rules
Det[NUM=sg,SEM=<\P Q.((DRS([x],[])+P(x))+Q(x))>] -> 'a' | 'A'
Det[NUM=sg,SEM=<\P Q.((DRS([x],[])+P(x))+Q(x))>] -> 'the' | 'The'
Adj[SEM=<\P x.(DRS([],[black(x)]) + P(x))>] -> 'black'
Adj[SEM=<\P x.(DRS([],[white(x)]) + P(x))>] -> 'white'
N[NUM=sg,SEM=<\x.DRS([],[kitten(x)])>] -> 'kitten'
N[NUM=sg,SEM=<\x.DRS([],[mouse(x)])>] -> 'mouse' 
N[NUM=sg,SEM=<\x.DRS([],[jar(x)])>] -> 'jar'
IV[NUM=sg,SEM=<\x.DRS([],[run_away(x)])>,tns=past] -> 'ran' 'away'
TV[SEM=<\X x.X(\y.DR

In [ ]:
grammar_str = """

% start S

# GRAMMAR RULES

S[SEM = <app(?subj,?vp)>] -> NP[NUM=?n,SEM=?subj] VP[NUM=?n,SEM=?vp]
NP[NUM=?n,SEM=<app(?det,?nom)> ] -> Det[NUM=?n,SEM=?det]  Nom[NUM=?n,SEM=?nom]
Nom[NUM=?n,SEM=?nom] -> N[NUM=?n,SEM=?nom]
Nom[NUM=?n,SEM=<?adj(?nom)>] -> Adj[SEM=?adj] N[NUM=?n,SEM=?nom] 
VP[NUM=?n,SEM=<app(?v,?obj)>] -> TV[NUM=?n,SEM=?v] NP[SEM=?obj]
VP[NUM=?n,SEM=?v] -> IV[NUM=?n,SEM=?v]


# LEXICAL RULES

# Articles
Det[NUM=sg,SEM=<\\P Q.(DRS([x],[]) + P(x) + Q(x))>] -> 'a' | 'A'
Det[NUM=sg,SEM=<\\P Q.((DRS([x],[PRO(x)]) + P(x)) + Q(x))>] -> 'the' | 'The'

# Nouns
N[NUM=sg,SEM=<\\x.DRS([],[kitten(x)])>] -> 'kitten'
N[NUM=sg,SEM=<\\x.DRS([],[mouse(x)])>] -> 'mouse' 
N[NUM=sg,SEM=<\\x.DRS([],[jar(x)])>] -> 'jar'

# Adjectives
Adj[SEM=<\\P x.(DRS([],[white(x)]) + P(x))>] -> 'white'
Adj[SEM=<\\P x.(DRS([],[black(x)]) + P(x))>] -> 'black'

# Verbs
TV[SEM=<\\X x.X(\\y.DRS([],[see(x,y)]))>,tns=past] -> 'saw'
TV[SEM=<\\X x.X(\\y.DRS([],[rush_after(x,y)]))>,tns=past] -> 'rushed' 'after'
TV[SEM=<\\X x.X(\\y.DRS([],[jump_into(x,y)]))>,tns=past] -> 'jumped' 'into'
TV[SEM=<\\X x.X(\\y.DRS([],[follow(x,y)]))>,tns=past] -> 'followed'
IV[SEM=<\\x.DRS([],[run_away(x)])>,tns=past] -> 'ran' 'away'
TV[SEM=<\\X x.X(\\y.DRS([],[get_our_of(x,y)]))>,tns=past] -> 'got' 'out' 'of'
"""

with open('my_grammar.fcfg', 'w') as f:
     f.write(grammar_str)

In [ ]:
our_expression = []
parser = load_parser('my_grammar.fcfg', logic_parser = nltk.sem.drt.DrtParser())

drs_list = []
for sent in target_sentences:
    drs_list.append(parse_p(parser, sent))
    print("The Target Sentence Is: ```", tsent, "```")
    print(drs_list[-1], '\n')

The Target Sentence Is: ``` A white kitten got out of the jar ```
([x,z430],[black(x), kitten(x), mouse(z430), see(x,z430)]) 

The Target Sentence Is: ``` A white kitten got out of the jar ```
([x,z434],[PRO(x), kitten(x), PRO(z434), mouse(z434), rush_after(x,z434)]) 

The Target Sentence Is: ``` A white kitten got out of the jar ```
([x,z438],[PRO(x), mouse(x), jar(z438), jump_into(x,z438)]) 

The Target Sentence Is: ``` A white kitten got out of the jar ```
([x,z442],[PRO(x), kitten(x), PRO(z442), mouse(z442), follow(x,z442)]) 

The Target Sentence Is: ``` A white kitten got out of the jar ```
([x],[PRO(x), mouse(x), run_away(x)]) 

The Target Sentence Is: ``` A white kitten got out of the jar ```
([x,z448],[white(x), kitten(x), PRO(z448), jar(z448), get_our_of(x,z448)]) 



In [ ]:
sumup_drs = drs_list[0]
for drs in drs_list[1:]:
    sumup_drs += drs
res = sumup_drs.simplify()
res_str = res.__str__()
res_str

'([x,z430,z434,z438,z442,z448,z449,z450,z451,z452,z453],[black(x), kitten(x), mouse(z430), see(x,z430), PRO(z449), kitten(z449), PRO(z434), mouse(z434), rush_after(z449,z434), PRO(z450), mouse(z450), jar(z438), jump_into(z450,z438), PRO(z451), kitten(z451), PRO(z442), mouse(z442), follow(z451,z442), PRO(z452), mouse(z452), run_away(z452), white(z453), kitten(z453), PRO(z448), jar(z448), get_our_of(z453,z448)])'

PRO Predicate

In [ ]:
resolution = [('z20', 'x'),  
              ('z12', 'z10'),
              ('z21', 'z10'),
              ('z22', 'x'),
              ('z16', 'z10'),
              ('z23', 'z10'),
              ('z19', 'z14')]
for r in resolution:
    pro = 'PRO(' + r[0] + ')'
    antecedent = r[1] + '=' + r[0]
    res_str = res_str.replace(pro, antecedent)
res_str

'([x,z430,z434,z438,z442,z448,z449,z450,z451,z452,z453],[black(x), kitten(x), mouse(z430), see(x,z430), PRO(z449), kitten(z449), PRO(z434), mouse(z434), rush_after(z449,z434), PRO(z450), mouse(z450), jar(z438), jump_into(z450,z438), PRO(z451), kitten(z451), PRO(z442), mouse(z442), follow(z451,z442), PRO(z452), mouse(z452), run_away(z452), white(z453), kitten(z453), PRO(z448), jar(z448), get_our_of(z453,z448)])'

In [ ]:
new_res = nltk.DrtExpression.fromstring(res_str)
new_res = new_res.eliminate_equality()
new_res

<DRS ([x,z430,z434,z438,z442,z448,z449,z450,z451,z452,z453],[black(x), kitten(x), mouse(z430), see(x,z430), PRO(z449), kitten(z449), PRO(z434), mouse(z434), rush_after(z449,z434), PRO(z450), mouse(z450), jar(z438), jump_into(z450,z438), PRO(z451), kitten(z451), PRO(z442), mouse(z442), follow(z451,z442), PRO(z452), mouse(z452), run_away(z452), white(z453), kitten(z453), PRO(z448), jar(z448), get_our_of(z453,z448)])>

In [ ]:
fol = new_res.fol()
fol

<ExistsExpression exists x z430 z434 z438 z442 z448 z449 z450 z451 z452 z453.(black(x) & kitten(x) & mouse(z430) & see(x,z430) & PRO(z449) & kitten(z449) & PRO(z434) & mouse(z434) & rush_after(z449,z434) & PRO(z450) & mouse(z450) & jar(z438) & jump_into(z450,z438) & PRO(z451) & kitten(z451) & PRO(z442) & mouse(z442) & follow(z451,z442) & PRO(z452) & mouse(z452) & run_away(z452) & white(z453) & kitten(z453) & PRO(z448) & jar(z448) & get_our_of(z453,z448))>

In [ ]:
print(len(drs_list))

6


In [ ]:
print(type(drs_list[0]))

<class 'nltk.sem.drt.DRS'>


In [ ]:
list_drs = drs_list[0]
for el in drs_list[1:]:
  list_drs += el

print(list_drs)

(([x,z430],[black(x), kitten(x), mouse(z430), see(x,z430)]) + ([x,z434],[PRO(x), kitten(x), PRO(z434), mouse(z434), rush_after(x,z434)]) + ([x,z438],[PRO(x), mouse(x), jar(z438), jump_into(x,z438)]) + ([x,z442],[PRO(x), kitten(x), PRO(z442), mouse(z442), follow(x,z442)]) + ([x],[PRO(x), mouse(x), run_away(x)]) + ([x,z448],[white(x), kitten(x), PRO(z448), jar(z448), get_our_of(x,z448)]))


In [ ]:
list_drs.simplify()

<DRS ([x,z430,z434,z438,z442,z448,z454,z455,z456,z457,z458],[black(x), kitten(x), mouse(z430), see(x,z430), PRO(z454), kitten(z454), PRO(z434), mouse(z434), rush_after(z454,z434), PRO(z455), mouse(z455), jar(z438), jump_into(z455,z438), PRO(z456), kitten(z456), PRO(z442), mouse(z442), follow(z456,z442), PRO(z457), mouse(z457), run_away(z457), white(z458), kitten(z458), PRO(z448), jar(z448), get_our_of(z458,z448)])>